In [1]:
import cv2
import numpy as np
import sklearn
import csv
from keras import optimizers
import matplotlib.pyplot as plt
from scipy.ndimage import rotate
%matplotlib inline
import random
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Dropout, Activation, GaussianNoise, ELU
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

Using TensorFlow backend.


In [ ]:
#generator to "create" a certain amount of pictures (batches)
#print_plt (bool) print out every picture with pyplot, for debugging
def generator(samples, batch_size=256, print_plt = False):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                center_image = random_brightness(cv2.imread(batch_sample[0]))
                if(batch_sample[2] == "flipped"):
                    center_image = cv2.flip(center_image, 1)
                angle = float(batch_sample[1])
                images.append(center_image)
                angles.append(angle)
                if(print_plt):
                    plt.figure()
                    plt.imshow(cv2.cvtColor(center_image, cv2.COLOR_BGR2RGB), cmap='gray')
                    

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

#print all the images with pyplot after they were processed by the generator
#images: (array of images) the images
def TestPrintPictures(images):
    imagesFromGenerator = generator(images, len(images), print_plt=True)
    im, y =next(imagesFromGenerator)

#bring the given array to the requested count, just approximated   
def BringToCount(imageArray, count):
    imagesTemp = sklearn.utils.shuffle(imageArray)
    newImageArray = []
    while len(newImageArray) < count:
        for image in imagesTemp:
            newImageArray.append(image.copy())
            if(len(newImageArray) > count):
                break
    return newImageArray

#for this project it is supposed that the straight direction picture overweigh. left and right will be increaed to a similar amount
def GetEqualShareLeftRightStraight(imageArray):
    notStraightAngle = 0.2
    imagesDict = {"left": [], "straight":[], "right":[]}
    pictureThrownOut = 0
    for i in range(len(imageArray)):
        if(imageArray[i][1] > -notStraightAngle and imageArray[i][1] < notStraightAngle):
            #ignore most pictures with steering angle 0, there are too many of them
            if(imageArray[i][1] == 0.0 and random.randint(1,3) > 1):
                pictureThrownOut = pictureThrownOut + 1
            else:
                imagesDict["straight"].append(imageArray[i])
        elif(imageArray[i][1] < -notStraightAngle):
            imagesDict["left"].append(imageArray[i])
        elif(imageArray[i][1] > notStraightAngle):
            imagesDict["right"].append(imageArray[i])
    print(pictureThrownOut, " picture with angle 0 thrown out.")
    lenRight = len(imagesDict["right"])
    lenStraight = len(imagesDict["straight"])
    lenLeft = len(imagesDict["left"])

    #assuming that straight has most images
    requestedAmount = int(lenStraight - ((lenStraight - lenRight) / 2))
    imagesDict["left"] = BringToCount(imagesDict["left"], requestedAmount)
    imagesDict["right"] = BringToCount(imagesDict["right"], requestedAmount)
    imagesDict["straight"] = BringToCount(imagesDict["straight"], requestedAmount)
    print("Count Images Left Steering: ", len(imagesDict["left"]))
    print("Count Images Straight: ", len(imagesDict["straight"]))
    print("Count Images Right Steering: ", len(imagesDict["right"]))
    returnArray = []
    returnArray.extend(imagesDict["right"])
    returnArray.extend(imagesDict["straight"])
    returnArray.extend(imagesDict["left"])

    return returnArray

#vary the brightness slightly to prevent overfitting
def random_brightness(image):
    brightness = np.random.uniform(0.4, 1.5)
    inv_gamma = 1.0 / brightness
    table = np.array([((i / 255.0) ** inv_gamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")

    return cv2.LUT(image, table)

#was used to create some documentation pictures            
def PrinImagesForDebugging(imageArray):
    imagesToPrint = []
    for image in imageArray:
        if(image[1] < -0.7 and image[2] == "normal" and image[3] == "center"):
            imagesToPrint.append(image)
            print("steering angle left ", image[1])
            break
    for image in imageArray:
        if(image[1] > -0.1 and image[1] < 0.1 and image[2] == "normal" and image[3] == "center"):
            imagesToPrint.append(image)
            print("steering angle center ", image[1])
            break
    for image in imageArray:
        if(image[1] > 0.7 and image[2] == "flipped" and image[3] == "center"):
            imagesToPrint.append(image)
            print("steering angle right ", image[1])
            break
    TestPrintPictures(imagesToPrint)
        

In [ ]:
#read entries from csv files with link to pictures, steering angle etc.
lines = []
images = []
batch_size = 192
with open("driving_log.csv") as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

print(len(lines), "entries found.")

for row in lines:
    steering_center = float(row[3])
    # read in images from center, left and right cameras
    img_center = ["IMG/" + row[0].split('\\')[-1], steering_center, "normal", "center"]
    img_left = ["IMG/" + row[1].split('\\')[-1], steering_center, "normal", "left"]
    img_right = ["IMG/" + row[2].split('\\')[-1], steering_center, "normal", "right"]
    img_center_flipped = ["IMG/" + row[0].split('\\')[-1], steering_center*-1.0, "flipped", "center"]
    #flipped, so right and left is exchanged
    img_left_flipped = ["IMG/" + row[1].split('\\')[-1], steering_center*-1.0, "flipped", "right"]
    img_right_flipped = ["IMG/" + row[2].split('\\')[-1], steering_center*-1.0, "flipped", "left"]

    # add images and angles to data set
    images.extend([img_center, img_left, img_right, img_center_flipped, img_left_flipped, img_right_flipped])
    
print(len(images), " imagepaths loaded")

images = GetEqualShareLeftRightStraight(images)
for image in images:
    # create adjusted steering measurements for the side camera images
    correction = 0.25 # this is a parameter to tune
    steering_center = float(image[1])

    if(image[3] == "left"):
        image[1] = image[1] + correction
        if(image[1] > 1):
            image[1] = 1
    if(image[3] == "right"):
        image[1] = image[1] - correction
        if(image[1] < -1):
            image[1] = -1
            
train_samples, validation_samples = train_test_split(images, test_size=0.1)


# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size)
validation_generator = generator(validation_samples, batch_size)

model = Sequential()
model.add(Lambda(lambda x: ((x/255.0)-0.5), input_shape = (160, 320, 3)))
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(3,160,320)))
#model.add(GaussianNoise(stddev=0.3))
model.add(Conv2D(24, (5, 5), strides=(2, 2),padding="valid"))
model.add(ELU())
model.add(Conv2D(32, (5, 5), strides=(2, 2),padding="valid"))
model.add(ELU())
model.add(Conv2D(48, (5, 5), strides=(2, 2),padding="valid"))
model.add(ELU())
model.add(Conv2D(64, (3, 3), strides=(1, 2),padding="valid"))
model.add(ELU())
model.add(Conv2D(64, (3, 3), strides=(1, 2),padding="valid"))
model.add(ELU())
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100))
model.add(ELU())
model.add(Dense(50))
model.add(ELU())
model.add(Dense(10))
model.add(ELU())
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.summary()
model.fit_generator(train_generator, epochs=2, validation_steps=len(validation_samples)/batch_size, steps_per_epoch=len(train_samples)/batch_size, validation_data=validation_generator)
model.save('model.h5')



20416 entries found.
122496  imagepaths loaded
8834  picture with angle 0 thrown out.
Count Images Left Steering:  50343
Count Images Straight:  50343
Count Images Right Steering:  50343
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_1 (Cropping2D)    (None, 90, 320, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 43, 158, 24)       1824      
_________________________________________________________________
elu_1 (ELU)                  (None, 43, 158, 24)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 77, 32)        19232     
_________________________________________________________________
elu_2 (ELU)          